In [1]:
import spacy
from spacy.matcher import Matcher
import json

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

def find_in_file(matcher, text):
    with open('../lab7/movies.json', 'r') as file:
        data = json.load(file)
    for dialogue in data[:30]:
        for turn in dialogue.get("turns", []):
            utterance = turn.get("utterance", "")
            doc = nlp(utterance)
            matches = matcher(doc)
            if matches:
                for _, start, end in matches:
                    span = doc[start:end]
                    print(f"Text: {utterance} \n {text} {span.text}")

patterns = [[{"POS": "PROPN", "OP": "+"},
            {"POS": "PROPN", "ENT_TYPE": "FAC"}, 
            {"IS_PUNCT": True, "TEXT": {"IN":[".", "!", "?"]}}],
            [{"POS": "PROPN", "ENT_TYPE": "FAC"}, 
            {"POS": "PROPN", "OP": "+"},
            {"IS_PUNCT": True, "TEXT": {"IN":[".", "!", "?"]}}]
]
matcher.add("CinemaNames", patterns)

find_in_file(matcher, "Cinema name found:")


Text: There's a showing at 10:30 pm at The New Parkway Theater. 
 Cinema name found: Parkway Theater.
Text: There's a showing at 10:30 pm at The New Parkway Theater. 
 Cinema name found: New Parkway Theater.
Text: There's one showing at 4:30 pm at the Vogue Theatre. 
 Cinema name found: Vogue Theatre.
Text: Is it on at any other times? I'd actually like to go to the Embarcadero Center Cinema. 
 Cinema name found: Center Cinema.
Text: Is it on at any other times? I'd actually like to go to the Embarcadero Center Cinema. 
 Cinema name found: Embarcadero Center Cinema.
Text: It's $14 per person, you can find it at 1 Embarcadero Center. 
 Cinema name found: Embarcadero Center.
Text: I want a 3D showing, on March 2nd, at the Vogue Theatre. 
 Cinema name found: Vogue Theatre.
Text: There is a 7:30 pm. showing at the Vogue Theatre. 
 Cinema name found: Vogue Theatre.
Text: I'd like to find a movie to watch at Century San Francisco Centre. 
 Cinema name found: Francisco Centre.
Text: I'd like 

In [2]:
confirmation_matcher = Matcher(nlp.vocab)

patterns = [
    [{"LOWER": "sounds"}, {"LOWER": "good"}],
    [{"LOWER": "that's"}, {"LOWER": "all"}, {"LOWER": "i"}, {"LOWER": "need"}],
    [{"LOWER": "yes"}, {"IS_PUNCT": True, "OP": "?"}],
    [{"LOWER": "that's"}, {"LOWER": {"IN": ["fine", "ok"]}}],
    [{"LOWER": "I'm"}, {"LOWER": "ok"}, {"LOWER": "with"}, {"LOWER": "that"}],
    [{"LOWER": "agree"}]
]
confirmation_matcher.add("Confirmations", patterns)

find_in_file(confirmation_matcher, "Confirmation found:")


Text: Sounds good, that's all I need. 
 Confirmation found: Sounds good
Text: Dr. Strangelove sounds good. I don't care about the theater, but what times are available? 
 Confirmation found: sounds good
Text: That sounds good. 
 Confirmation found: sounds good
Text: Red Joan sounds good, what time is it at? 
 Confirmation found: sounds good
Text: Citizen Kane sounds good. 
 Confirmation found: sounds good
Text: That sounds good. 
 Confirmation found: sounds good
Text: Yes, found 3 movies suit your taste. Little, Long shot and Missing link, which one is interested to you? 
 Confirmation found: Yes
Text: Yes, found 3 movies suit your taste. Little, Long shot and Missing link, which one is interested to you? 
 Confirmation found: Yes,
Text: That sounds good. 
 Confirmation found: sounds good
Text: Hellboy sounds good. 
 Confirmation found: sounds good
Text: Yes. When can I see Teen Spirit 
 Confirmation found: Yes
Text: Yes. When can I see Teen Spirit 
 Confirmation found: Yes.
Text: yes 

In [3]:
def extract_intents(doc):
    intents = []
    for token in doc:
        if token.dep_ == "dobj":
            intent = (token.head.text, token.text)
            intents.append(intent)
    return intents

with open('../lab7/movies.json', 'r') as file:
    data = json.load(file)

for entry in data[:30]:
    for turn in entry.get("turns", []):
        utterance = turn.get("utterance", "")
        doc = nlp(utterance)
        intents = extract_intents(doc)
        if len(intents) > 0:
            for intent in intents:
                print(f"Text: {utterance}")
                print(f"Intent found: Action - {intent[0]}, Object - {intent[1]}")


Text: A movie will help me overcome the monotonous of the day. Is there any good movie?
Intent found: Action - overcome, Object - monotonous
Text: There is Ash Is Purest White, Body Double and Captain Marvel, which one do you prefer?
Intent found: Action - prefer, Object - one
Text: Do you like Citizen Kane, Dogman, or Dumbo?
Intent found: Action - like, Object - Kane
Text: Try some other particularly in Vallejo, imax show.
Intent found: Action - Try, Object - other
Text: Will Hellboy please you?
Intent found: Action - please, Object - you
Text: Thats great, Hellboy will do. Show time please
Intent found: Action - Show, Object - time
Text: Which date do you prefer?
Intent found: Action - prefer, Object - date
Text: Do you know the genre of movie and fare of ticket.
Intent found: Action - know, Object - genre
Text: Yeah, I have changed the date to March 1st and at Century Vallejo.
Intent found: Action - changed, Object - date
Text: Have a great day.
Intent found: Action - Have, Object -